# 371 - Licence Plates

## Problem Statement

<p>
Oregon licence plates consist of three letters followed by a three digit number (each digit can be from [0..9]).<br>
While driving to work Seth plays the following game:<br>
Whenever the numbers of two licence plates seen on his trip add to 1000 that's a win.
</p>
<p>
E.g. MIC-012 and HAN-988 is a win and RYU-500 and SET-500 too (as long as he sees them in the same trip). 
</p><p>
</p><p>
Find the expected number of plates he needs to see for a win.<br>
Give your answer rounded to 8 decimal places behind the decimal point.
</p>
<p style="font-size:88%;">
<b>Note:</b> We assume that each licence plate seen is equally likely to have any three digit number on it.
</p>

## Solution

First, we note that letters are irrelevant to the problem so we will focus on the numbers only.

Then, we can make a few observations regarding the numbers that Seth encounters:
- the plate number 000 cannot be paired with another plate number to sum to 1000,
- the plate number 500 must be paired with another plate number 500 to sum to 1000 and it is the only plate number that will lead to a win if it is seen twice, 
- any other number $x \neq 500$ needs to be paired with a different number $y = 1000 - x$ to win.

This means that Seth can see at most 501 different plate numbers before winning.

We can model this using a Markov chain. Let $E_{i, j}$ denote the expected number of cars Seth needs to see to win from a state $(i, j)$ where he has encountered $i$ plates different from 000 and 500 and $j$ is a binary variable set to 1 if we have already encountered the plate number 500 and 0 otherwise. What we are interested in is $E_{0, 0}$ which is the expected number of cars we need to see when we have not see any car so far. Let $W$ denote the state where Seth wins the game. Once the game is won, the expected number of cars needed to win the game is zero so $E_W = 0$. Initially Seth has not seen any licence plate so he is in state (0, 0). There are 3 possibilities when he sees a car. With probability 1/1000, Seth sees a 000 plate number and stays in the same state. With probability 1/1000, he sees a 500 plate number and moves to state (0, 1). Otherwise, he moves to state (1, 0) with probability 998/1000. In all those cases, he will have seen one additional car. Therefore we have

\begin{equation}
    E_{0, 0} = \frac{1}{1000} E_{0, 0} + \frac{1}{1000} E_{0, 1} + \frac{998}{1000} E_{1, 0} + 1.
\end{equation}

Without loss of generality, we will assume that Seth always meets new plate numbers different from 000 and 500 in increasing order (i.e., in state (1, 0) he has seen plate number 001, in state (2, 0) he has seen plates 001 and 002, etc). Let's say Seth is now in state (1, 0) meaning that he has encountered the 001 plate number. If the next car has plate number 000 or 001, happening with probability 2/1000, he will stay in the same state. If the next car has plate number 500, happening with probability 1/1000, he will move to state (1, 1). If the plate number is 999, happening with probability 1/1000, he will win the game and move to state $W$. Otherwise, he sees a new plate number and moves to state (2, 0) and this happens with probability 996/1000. Again he has seen one additional car so we have

\begin{align}
    E_{1, 0} &= \frac{2}{1000} E_{1, 0} + \frac{1}{1000} E_{1, 1} + \frac{1}{1000} E_W + \frac{996}{1000} E_{2, 0} + 1 \\
            &= \frac{2}{1000} E_{1, 0} + \frac{1}{1000} E_{1, 1} + \frac{996}{1000} E_{2, 0} + 1.
\end{align}

If instead Seth first sees the plate number 500, he would be in state (0, 1). In this case, he only stays in the same state if he sees the plate number 000 (with probability 1/1000). If he sees the plate number 500, he wins the game and this happens with probability 1/1000. Otherwise, we probability 998/1000, he sees a new plate number and moves to state (1, 1). The equation is

\begin{align}
    E_{0, 1} &= \frac{1}{1000} E_{0, 1} + \frac{1}{1000} E_W + \frac{998}{1000} E_{1, 1} + 1 \\
            &= \frac{1}{1000} E_{0, 1} + \frac{998}{1000} E_{1, 1} + 1.
\end{align}

Continuing with the same logic, the final state where all the plate numbers have been seen once would be

\begin{align}
    E_{499, 1} &= \frac{500}{1000} E_{499, 1} + \frac{500}{1000} E_W + 1 \\
            &= \frac{1}{2} E_{499, 1} + 1 \\
            &= 2.
\end{align}

We now work backward solving all the equations until we can solve for $E_{0, 0}$. We note that for $i < 499$, we have

\begin{align}
    E_{i, 1} &= \frac{1000}{1000 - i - 1} \left(\frac{1000 - 2i - 2}{1000} E_{i+1, 1} + 1 \right) \\
            &= \frac{(1000 - 2i - 2)E_{i+1, 1} + 1000}{1000 - i - 1},
\end{align}

and 

\begin{align}
    E_{i, 0} &= \frac{1000}{1000 - i - 1} \left(\frac{1}{1000} E_{i, 1} + \frac{1000 - 2i - 2}{1000} E_{i+1, 0} + 1 \right) \\
            &= \frac{E_{i, 1} + (1000 - 2i - 2)E_{i+1, 0} + 1000}{1000 - i - 1}.
\end{align}

In [1]:
# e1 indicates 500 has been seen and e0 indicates 500 has not been seen
e1 = e0 = 0
for i in range(499, -1, -1):
    e1 = ((1000 - (2*i + 2)) * e1 + 1000) / (1000 - i - 1)
    e0 = (e1 + (1000 - (2*i + 2)) * e0 + 1000) / (1000 - i - 1)

round(e0, 8)

40.66368097